## Apply to MSWEP

In [1]:
from dask.distributed import LocalCluster, Client, progress
cluster = LocalCluster(ip="0.0.0.0", threads_per_worker=2, n_workers=16)
client = Client(cluster)
#client = Client(ip="0.0.0.0", processes=True, threads_per_worker=8,
#               n_workers=4, memory_limit='8GB'
cluster

2023-02-02 10:30:30,454 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-0ld0ynkg', purging
2023-02-02 10:30:30,454 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-72haey9n', purging
2023-02-02 10:30:30,455 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-wk4976vu', purging
2023-02-02 10:30:30,455 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-s8104_ud', purging
2023-02-02 10:30:30,455 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-j8_24_54', purging
2023-02-02 10:30:30,455 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-i_12ppxp', purging
2023-02-02 10:30:30,455 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-_vevatxo', purging

LocalCluster(29aee674, 'tcp://136.172.124.6:43801', workers=16, threads=32, memory=503.45 GiB)

In [5]:
import kerchunk
import xarray as xr
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import fsspec
from glob import glob

In [6]:
def gen_ref(f):
    with fsspec.open(f, mode="rb", anon=True) as infile:
        return SingleHdf5ToZarr(infile, f, inline_threshold=300).translate()

In [7]:
indir="/work/bb1153/b382075/datasets/MSWEP_V280/Past/Monthly"

In [5]:
fs = fsspec.filesystem('file')
flist = fs.glob(indir + '/' + '*.nc')
bag = db.from_sequence(flist, npartitions=len(flist)).map(gen_ref)
%time dicts = bag.compute()

In [ ]:
mzz = MultiZarrToZarr(
        dicts,
        concat_dims='time',
        inline_threshold=0
    )
%time mzz.translate(indir + f'/../../mswep_past_monthly_new.json')

In [7]:
n=len(flist)
istep=2048

In [9]:
import numpy as np
np.arange(0, n, istep)

array([    0,  2048,  4096,  6144,  8192, 10240, 12288, 14336])

In [10]:
import numpy as np
import dask.bag as db
n=len(flist)
istep=2048
dicts = []
j=5
for i in np.arange(10240, n, istep):
    imax = min(n, i + istep)
    print(i, imax)
    bag = db.from_sequence(flist[i:imax], npartitions=64).map(gen_ref)
    %time d = bag.compute()
    mzz = MultiZarrToZarr(
        d,
        concat_dims='time',
        inline_threshold=0
    )
    %time mzz.translate(indir + f'/../../mswep_past_daily{j}.json')
    dicts = dicts + d
    j = j + 1

10240 12288
CPU times: user 25.8 s, sys: 4.5 s, total: 30.3 s
Wall time: 2min 36s
CPU times: user 2min 24s, sys: 1min 2s, total: 3min 26s
Wall time: 2min 50s
12288 14336
CPU times: user 27.6 s, sys: 6.41 s, total: 34 s
Wall time: 2min 40s
CPU times: user 2min 23s, sys: 1min 2s, total: 3min 25s
Wall time: 2min 50s
14336 15339
CPU times: user 13.9 s, sys: 4.28 s, total: 18.1 s
Wall time: 1min 21s
CPU times: user 1min 8s, sys: 27.1 s, total: 1min 35s
Wall time: 1min 19s


In [8]:
len(dicts)

15339

In [3]:
import pickle

with open(indir + '/../../dicts.pickle', 'wb') as handle:
    pickle.dump(dicts, handle, protocol=pickle.HIGHEST_PROTOCOL)


NameError: name 'dicts' is not defined

In [4]:
with open(indir + '/../../dicts.pickle', 'rb') as handle:
    dicts = pickle.load(handle)

EOFError: Ran out of input

In [9]:
mzz = MultiZarrToZarr(
    dicts,
    # sorted((glob('./example_jsons/individual/*.json'))),
    concat_dims='time',
    inline_threshold=0
)

In [10]:
%time mzz.translate(indir + '/../../mswep_past_daily.json')

: 

: 

### Reading the data

In [5]:
fs = fsspec.filesystem(
    'reference', 
    fo=indir + '/../../mswep_past_daily0.json',  
    skip_instance_cache=True
)

In [6]:
ds = xr.open_dataset(fs.get_mapper(""), engine='zarr')
ds

/tmp/ipykernel_3048561/1456563027.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds = xr.open_dataset(fs.get_mapper(""), engine='zarr')


<xarray.Dataset>
Dimensions:        (lat: 1800, lon: 3600, time: 2048)
Coordinates:
  * lat            (lat) float32 89.95 89.85 89.75 ... -89.75 -89.85 -89.95
  * lon            (lon) float32 -179.9 -179.8 -179.8 ... 179.8 179.9 179.9
  * time           (time) datetime64[ns] 1979-01-02 1979-01-03 ... 1984-08-10
Data variables:
    precipitation  (time, lat, lon) float32 ...
Attributes:
    history:  Created on 2021-02-03 20:47

In [48]:
ds2 = xr.open_mfdataset(indir + '/' + '*.nc', engine='netcdf4')